In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 95% !important; }</style>"))

In [ ]:
import sys
import os
import re
import numpy as np
import pandas as pd

# from pathlib import Path
from tqdm.notebook import tqdm

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
from lib import util, embeds, fitter

In [ ]:
# FIXME: add lines for git clone SemeEval2022 and Stitch data

In [ ]:
datapath = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data'

Load all the CSV files in dataframes.

In [ ]:
frames = util.load_csv_dataframes(datapath)

In [ ]:
frames.keys()

In [ ]:
zdf = frames['train_zero_shot.csv']
odf = frames['train_one_shot.csv']
ddf = frames['dev.csv']
ddf_gold = frames['dev_gold.csv']
edf = frames['eval.csv']

In [ ]:
z_emb = embeds.get_embeddings(zdf)
z_emb_i = embeds.get_embeddings(zdf, append=['MWE'])

In [ ]:
multilingual_model = 'distiluse-base-multilingual-cased-v1'

Get sentence-transformers embeddings with the best method (appending MWE to the text, ignoring context).

The "best" method isn't actually completely true, as the original paper uses the "idiomatic principle" to encode the MWE, that is, using it as a single token when tokenizing.

In [ ]:
z_emb_multi = embeds.get_embeddings(zdf, modelname=multilingual_model, append=['MWE'])

In [ ]:
d_emb_multi = embeds.get_embeddings(ddf, modelname=multilingual_model, append=['MWE'])

Do a fitting with Logistic Regression.

In [ ]:
z_score, z_probs, z_results = fitter.get_fit_results(z_emb_multi, zdf['Label'], d_emb_multi, ddf_gold['Label'])

In [ ]:
z_score

In [ ]:
dres = fitter.add_results(ddf, z_results, ddf_gold)

In [ ]:
dres

In [ ]:
dres_counts = util.get_counts(dres)

Show the MWEs that the model gets wrong more than half of the time. Are there any patterns?

In [ ]:
dres_counts[dres_counts['Pct correct'] < 0.5].sort_values(by=['Language','MWE'])